In [1]:
import d2lzh as d2l
import math
from mxnet import autograd, nd
from mxnet.gluon import loss as gloss
import time

(corpus_indices, char_to_idx, idx_to_char,
 vocab_size) = d2l.load_data_jay_lyrics()

In [2]:
# 索引为0和2的one-hot向量
nd.one_hot(nd.array([0, 2]), vocab_size)


[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]
<NDArray 2x1027 @cpu(0)>

In [3]:
def to_onehot(X, size):
    return [nd.one_hot(x, size) for x in X.T]

X = nd.arange(10).reshape((2,5))
inputs = to_onehot(X, vocab_size)
len(inputs), inputs[0].shape

(5, (2, 1027))

In [4]:
inputs

[
 [[1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]
 <NDArray 2x1027 @cpu(0)>,
 
 [[0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]
 <NDArray 2x1027 @cpu(0)>,
 
 [[0. 0. 1. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]
 <NDArray 2x1027 @cpu(0)>,
 
 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]
 <NDArray 2x1027 @cpu(0)>,
 
 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]
 <NDArray 2x1027 @cpu(0)>]

In [5]:
X


[[0. 1. 2. 3. 4.]
 [5. 6. 7. 8. 9.]]
<NDArray 2x5 @cpu(0)>

In [6]:
# num_hiddens是一个超参数
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
ctx = d2l.try_gpu()
print('will use', ctx)


will use gpu(0)


In [16]:
def get_params():
    def _one(shape):
        return nd.random.normal(scale=0.01, shape=shape, ctx=ctx)
    
    # 隐藏层参数
    W_xh_1 = _one((num_inputs, num_hiddens))
    W_hh_1 = _one((num_hiddens, num_hiddens))
    b_h_1 = nd.zeros(num_hiddens, ctx=ctx)
    W_xh_2 = _one((num_hiddens, num_hiddens))
    W_hh_2 = _one((num_hiddens, num_hiddens))
    b_h_2 = nd.zeros(num_hiddens, ctx=ctx)
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = nd.zeros(num_outputs, ctx=ctx)
    # 附上梯度
    params = [W_xh_1, W_hh_1, b_h_1, W_xh_2, W_hh_2, b_h_2, W_hq, b_q]
    for param in params:
        param.attach_grad()
    return params

In [17]:
# 返回初始化的隐藏状态
def init_rnn_state(batch_size, num_hiddens, ctx):
    return (nd.zeros(shape=(batch_size, num_hiddens), ctx=ctx), 
            nd.zeros(shape=(batch_size, num_hiddens), ctx=ctx))

In [18]:
# 循环神经网络模型，relu激活函数
def rnn(inputs, state, params):
    # inputs和outputs皆为num_steps个形状为(batch_size, vocab_size)的矩阵
    W_xh_1, W_hh_1, b_h_1, W_xh_2, W_hh_2, b_h_2, W_hq, b_q = params
    H_1,H_2 = state
    outputs = []
    for X in inputs:
        H_1 = nd.relu(nd.dot(X, W_xh_1) + nd.dot(H_1, W_hh_1) + b_h_1)
        H_2 = nd.relu(nd.dot(H_1, W_xh_2) + nd.dot(H_2, W_hh_2) + b_h_2)
        Y = nd.dot(H_2, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H_1,H_2)

In [22]:
state = init_rnn_state(X.shape[0], num_hiddens, ctx)
inputs = to_onehot(X.as_in_context(ctx), vocab_size)
params = get_params()
outputs, state_new = rnn(inputs, state, params)
len(outputs), outputs[0].shape, state_new[0].shape, state_new[1].shape

(5, (2, 1027), (2, 256), (2, 256))

In [23]:
def predict_rnn(prefix, num_chars, rnn, params, init_rnn_state, num_hiddens, vocab_size, ctx, idx_to_char, char_to_idx):
    state = init_rnn_state(1, num_hiddens, ctx)
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars + len(prefix) -  1):
        # 将上一时间步的输出作为当前时间步的输入
        X = to_onehot(nd.array([output[-1]], ctx=ctx), vocab_size)
        # 计算输出和更新隐藏状态
        (Y, state) = rnn(X, state, params)
        # 下一个时间步的输入是prefix里的字符或者当前的最佳预测字符
        if t < len(prefix) -1:
            output.append(char_to_idx[prefix[t+1]])
        else:
            output.append(int(Y[0].argmax(axis=1).asscalar()))
    return ''.join([idx_to_char[i] for i in output])

In [24]:
predict_rnn('分开', 10, rnn, params, init_rnn_state, num_hiddens, vocab_size,
            ctx, idx_to_char, char_to_idx)

'分开可骨盯死鸦息息息息息'

In [25]:
# 循环神经网络中较容易出现梯度衰减或梯度爆炸。
# 裁剪梯度（clip gradient）
def grad_clipping(params, theta, ctx):
    norm = nd.array([0], ctx)
    for param in params:
        norm += (param.grad **2).sum()
    norm = norm.sqrt().asscalar()
    # 把所有模型参数梯度的元素拼接成一个向量g，也就是下面的norm
    if norm > theta:
        for param in params:
            param.grad[:] *= theta/norm

In [26]:
# 本函数已保存在d2lzh包中方便以后使用
def train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                          vocab_size, ctx, corpus_indices, idx_to_char,
                          char_to_idx, is_random_iter, num_epochs, num_steps,
                          lr, clipping_theta, batch_size, pred_period,
                          pred_len, prefixes):
    if is_random_iter:
        data_iter_fn = d2l.data_iter_random
    else:
        data_iter_fn = d2l.data_iter_consecutive
    params = get_params()
    loss = gloss.SoftmaxCrossEntropyLoss()

    for epoch in range(num_epochs):
        if not is_random_iter:  # 如使用相邻采样，在epoch开始时初始化隐藏状态
            state = init_rnn_state(batch_size, num_hiddens, ctx)
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_fn(corpus_indices, batch_size, num_steps, ctx)
        for X, Y in data_iter:
            if is_random_iter:  # 如使用随机采样，在每个小批量更新前初始化隐藏状态
                state = init_rnn_state(batch_size, num_hiddens, ctx)
            else:  # 否则需要使用detach函数从计算图分离隐藏状态
                for s in state:
                    s.detach()
            with autograd.record():
                inputs = to_onehot(X, vocab_size)
                # outputs有num_steps个形状为(batch_size, vocab_size)的矩阵
                (outputs, state) = rnn(inputs, state, params)
                # 连结之后形状为(num_steps * batch_size, vocab_size)
                outputs = nd.concat(*outputs, dim=0)
                # Y的形状是(batch_size, num_steps)，转置后再变成长度为
                # batch * num_steps 的向量，这样跟输出的行一一对应
                y = Y.T.reshape((-1,))
                # 使用交叉熵损失计算平均分类误差
                l = loss(outputs, y).mean()
            l.backward()
            grad_clipping(params, clipping_theta, ctx)  # 裁剪梯度
            d2l.sgd(params, lr, 1)  # 因为误差已经取过均值，梯度不用再做平均
            l_sum += l.asscalar() * y.size
            n += y.size

        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn(
                    prefix, pred_len, rnn, params, init_rnn_state,
                    num_hiddens, vocab_size, ctx, idx_to_char, char_to_idx))

In [27]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']

In [28]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, ctx, corpus_indices, idx_to_char,
                      char_to_idx, True, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 83.970037, time 0.47 sec
 - 分开 我想的让我疯狂  爱想的 你不是 我爱你你 你不的 我不你你 你你的 我不你你不 我不你你 你不的
 - 不分开 我想的 你不是 我说的 你不你 你说的 我不你你 你你的 我不你你不 我不你你 你不的 我不你你 
epoch 100, perplexity 15.993566, time 0.46 sec
 - 分开 我想你这爱  不知话不 我不好努 你你的带 我 想和你的 我不要再想 我不 我不 我不 我不 我不
 - 不分开 我想你这不  不知后觉 你该好美 是谁在练 是不定带 你你想 我不 我不 我不 我不 我不 我不 
epoch 150, perplexity 4.427812, time 0.46 sec
 - 分开 我想就够样牵着  这想你的 我想要将我的寂对 消失说 旧皮我 没想大声 沙故的 告想我都不经袋 说
 - 不分开 我想动这样牵 我以都神到  也悔着对不 我不要再想 我不要再想 我不 我不 我不要 爱情走的太快就
epoch 200, perplexity 2.140519, time 0.52 sec
 - 分开 我想就够样牵着你的手不放开 爱可不可以简简单单没有伤想 你满就风诅 别窝在 小弄堂 装满了明信片的
 - 不分开 快只两颗担忧 唱着歌 一直走 我想就这样牵着你的手不放开 爱可不可以简简单单没有伤想 你满就风诅 
epoch 250, perplexity 1.655229, time 0.47 sec
 - 分开落 时伤得很隐密 它在许愿池里轻轻叹息 太多的我爱你 让它喘不过气 不经 失去意义 戒指在哭泣 静静
 - 不分开初 你一子到 你 想上你骑睡车 想要下 说 我 开在                        


In [71]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, ctx, corpus_indices, idx_to_char,
                      char_to_idx, False, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 63.521017, time 0.31 sec
 - 分开 我不能这样 三色人  我已了这 你谁一空 你 一直 我想定你 你有一空 你谁一空 你 一直 我想定
 - 不分开 我想我这样 谁不人   我的外 你天了空 你谁了空 你谁一空 你 一直 你 一场我 别你的 娘颗我
epoch 100, perplexity 8.488997, time 0.31 sec
 - 分开 娘子 是谁B 后想走 干什么 旧想开 旧满了 我想店这样 三两 什不 你的 我感动 想爱走人 如过
 - 不分开 我想能会呵 三两银够不够 景色入秋 漫天黄沙截记 哼哼哈兮 快使用双截记 哼哼哈兮 快使用双截记 
epoch 150, perplexity 2.373217, time 0.31 sec
 - 分开 什么我 谁是神枪手久白墙我叫泪一声 我说定很天的黑样 古会什么我想要 却发凶迷了直到我 也物我的在
 - 不分开只 你已经 开怎么 什么都有 周杰事 的片段 时一些风霜 老唱盘 旧皮箱 装属了明信片的铁盒里藏著一
epoch 200, perplexity 1.621490, time 0.31 sec
 - 分开 她养的黑猫笑起来像哭 啦啦啦呜 用水晶球替人占卜 她说下午三点阳光射道 就是那么简单几句 我办不到
 - 不分开觉 我已那离 小小不外的溪边 情绪激动 一颗心到现在还在抽痛 还为分手前那句抱歉 在感动 穿梭时间的
epoch 250, perplexity 1.323210, time 0.31 sec
 - 分开 问养我 谁是神枪手 巫师 他念念 有词的 对酋长下诅咒 还我骷髅头 这故事 告诉我 印地安的传说 
 - 不分开觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生


In [29]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, ctx, corpus_indices, idx_to_char,
                      char_to_idx, False, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 87.366536, time 0.47 sec
 - 分开 我感动  爱场人                                         
 - 不分开  你    我                                          
epoch 100, perplexity 15.058095, time 0.47 sec
 - 分开 娘子却到不江南等我 泪著你其语背久定永现一到 一情上 干什么 干什么 干你我 说你我 说你怎么不球
 - 不分开 说要上这 我不已乡我的你我的双道 我不在在过 我将无伊斯写 唱彻不 一皮走 一颗两颗三颗四颗 连成
epoch 150, perplexity 3.452952, time 0.47 sec
 - 分开 娘子却人在每日 我一枝杨柳 你在那里 在小村外的溪边河口默默等著我 娘子一口每著 像这念愿国坦 闪
 - 不分开 说你的爹娘不擅 从才方向 如果这一切 白色人烛 我该经一个我 选你线进球 单手过人运球 篮下里传出
epoch 200, perplexity 1.703367, time 0.47 sec
 - 分开的在会 干什么这个人 还的话一球我 抛物线进球 单手过人运球 篮下妙传著手 快攻的篮板人 得分了最原
 - 不分开 说么兵器最喜欢 双截棍柔中带刚 想要去河南嵩山 学少林跟武当 快使用双截棍 哼哼哈兮 如果用有切功
epoch 250, perplexity 1.274664, time 0.49 sec
 - 分开的在打 还真得一城堡 它爱拥有的只剩下回忆 相爱还有别离 像无法被安排的雨 随时准备来袭 我怀念起国
 - 不分开 说是我较样打我妈妈 我说的话你甘会听 不要再这样打我妈妈 难道你手不会痛吗 其实我回家就想要阻止一
